<a href="https://colab.research.google.com/github/andyleva/natural-language-processing/blob/lesson05/nlp_hw_lesson_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### КУРС "Введение в обработку естественного языка"

дз к уроку «POS-tagger и NER»

Задание 1. Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные
векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы, сделать выводы
Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.
a. передаём в сетку токен и его соседей.
b. передаём в сетку только токен.
c. свой вариант.
3. сравнить свои реализованные подходы на качество — вывести
precision/recall/f1_score.

In [ ]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.5 MB/s 
     |████████████████████████████████| 8.2 MB 40.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=fcabbb088f8b13fe28b237975a6314b19b1c3ff4cf8ad5309c92cb33f83e5314
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
from pymorphy2 import MorphAnalyzer
from razdel import tokenize, sentenize
import corus
from corus import load_rudrec
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.tag import SequentialBackoffTagger
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import pyconll
!wget -O ./ru_syntagrus-ud-train.conllu https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./ru_syntagrus-ud-test.conllu https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu

--2022-12-06 18:48:53--  https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-train-a.conllu
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu [following]
--2022-12-06 18:48:53--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘./ru_syntagrus-ud-train.conllu’

./ru_syntagrus-ud-t 100%[===================>]  38.85M  --.-KB/s    in 

In [ ]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-test.conllu')

In [ ]:
# view data 
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)

Анкета NOUN
. PUNCT
Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT


In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [ ]:
len(fdata_train), len(fdata_sent_test)

(24516, 8800)

In [ ]:
# see what is data
fdata_test[2]

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'VERB'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', 'ADJ'),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', 'PROPN'),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', 'VERB'),
 ('добывающие', 'VERB'),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', 'NOUN'),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

In [ ]:
# view how tags most valued
tags = [sent[0][1] for sent in fdata_train]
nltk.FreqDist(tags).most_common()

[('ADP', 3831),
 ('NOUN', 3396),
 ('ADV', 3356),
 ('CCONJ', 2652),
 ('ADJ', 2130),
 ('PRON', 2108),
 ('VERB', 1781),
 ('PUNCT', 1677),
 ('PART', 862),
 ('DET', 846),
 ('SCONJ', 820),
 ('PROPN', 650),
 ('NUM', 330),
 ('AUX', 47),
 ('INTJ', 21),
 ('X', 5),
 (None, 4)]

In [ ]:
# test DefaultTagger
default_tagger = nltk.DefaultTagger('NOUN')
display(default_tagger.tag(fdata_sent_test[2]), default_tagger.evaluate(fdata_test))


[('С', 'NOUN'),
 ('одной', 'NOUN'),
 ('стороны', 'NOUN'),
 (',', 'NOUN'),
 ('квалифицированные', 'NOUN'),
 ('кадры', 'NOUN'),
 ('и', 'NOUN'),
 ('развитая', 'NOUN'),
 ('производственная', 'NOUN'),
 ('инфраструктура', 'NOUN'),
 ('резко', 'NOUN'),
 ('отличали', 'NOUN'),
 ('Армению', 'NOUN'),
 ('от', 'NOUN'),
 ('других', 'NOUN'),
 ('регионов', 'NOUN'),
 ('СССР', 'NOUN'),
 (',', 'NOUN'),
 ('где', 'NOUN'),
 ('доминировали', 'NOUN'),
 ('добывающие', 'NOUN'),
 ('отрасли', 'NOUN'),
 (',', 'NOUN'),
 ('а', 'NOUN'),
 ('экономика', 'NOUN'),
 ('строилась', 'NOUN'),
 ('на', 'NOUN'),
 ('поставке', 'NOUN'),
 ('сырьевых', 'NOUN'),
 ('ресурсов', 'NOUN'),
 ('.', 'NOUN')]

0.23145915221945831

In [ ]:
# now look at UnigramTagger
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[2]), unigram_tagger.evaluate(fdata_test))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.8369316851173183

In [ ]:
# made bigram_tagger on basis unigram_tagger
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[2]), bigram_tagger.evaluate(fdata_test))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.8419889992341238

In [ ]:
# build trigram_tagger on basis unigramm_tagger
trigram_tagger_unigram = TrigramTagger(fdata_train, backoff=unigram_tagger)
display(trigram_tagger_unigram.tag(fdata_sent_test[2]), trigram_tagger_unigram.evaluate(fdata_train))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.9848093068219681

In [ ]:
# build trigram_tagger on basis bigramm_tagger
trigram_tagger_unigram = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger_unigram.tag(fdata_sent_test[2]), trigram_tagger_unigram.evaluate(fdata_train))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.984957130990985

комбинация разных taggers

In [ ]:
def combine_tagger(train_data, tagger_class, backoff=None):
    for cls in tagger_class:
        backoff = cls(train_data, backoff = backoff)
    
    return backoff

backoff = DefaultTagger('NOUN')
tagger = combine_tagger(fdata_train, [UnigramTagger, BigramTagger, TrigramTagger], backoff = backoff)
tagger.evaluate(fdata_test)

0.8893720448892012

свой собственный тэггер

In [ ]:
propn = []
morph = MorphAnalyzer()

for sent in fdata_train:    
    for word in sent:
        if word[1] == 'PROPN':
            new_propn = morph.normal_forms(word[0].lower())[0]
            if new_propn not in propn:
                propn.append(new_propn)
propn[:5]

['семён', 'еремеевич', 'семя', 'ефимов', 'москва']

In [ ]:
class CustomTagger(SequentialBackoffTagger):
    def __init__(self, *arg, **kwargs):
        SequentialBackoffTagger.__init__(self, *arg, **kwargs)
        self.tags_set = set([n for n in propn])
        
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.tags_set:
            return 'TARGET'
        else:
            return None

Работа с NER

In [ ]:
#test_article = [line for line in file()]
test_article = ""

with open('./sample_data/last_72.txt') as file:
    test_article = [line for line in file]
test_article = '\n'.join(test_article)
test_article = test_article.replace('\n','')
test_article = test_article.replace('Людмила', 'Людмила Хохрякова')
test_article

'На Ставрополье создан Общественный совет при МинэкономразвитияПри министерстве экономического развития Ставропольского края в рамках проекта "открытое правительство" создан Общественный совет, сообщили корреспонденту ИА REGNUM в пресс-службе краевого Минэкономразвития 23 июля.Как отметила на организационном заседании совета первый заместитель министра Людмила Хохрякова Хохрякова, Общественный совет должен будет решать задачи создания общественно-государственного партнерства, взаимодействия власти и общества."Нам бы хотелось услышать от вас конкретные предложения по улучшению социально-экономического развития края, по фактическому применению в деятельности министерства современных научных разработок для повышения эффективности и достижения ряда экономических показателей; рекомендации по осуществлению и корректировке государственной политики в сфере развития инвестиционной и инновационной деятельности, малого и среднего предпринимательства, снижению административных барьеров", - отметил

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_article))) if hasattr(chunk, 'label') }

{('Людмила Хохрякова Хохрякова', 'PERSON'),
 ('Михаил Зверев', 'PERSON'),
 ('Николай Иванов', 'PERSON'),
 ('Общественный', 'PERSON'),
 ('Совета', 'PERSON'),
 ('Ставропольские', 'PERSON')}

In [ ]:
pip install ipymarkup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26118 sha256=0599322a6c4015614201b8e483e2cc0a17a4707a0788ca380a9e0861fb15da1d
  Stored in directory: /root/.cache/pip/wheels/45/23/de/5789a92962483fd33cb06674792b9697c1b3766d7c7742830e
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [ ]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2022-12-06 19:18:04--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  18.4MB/s    in 2.7s    

2022-12-06 19:18:07 (18.4 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
!wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar

--2022-12-06 19:22:55--  https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2385920 (2.3M) [application/octet-stream]
Saving to: ‘slovnet_ner_news_v1.tar’

slovnet_ner_news_v1 100%[===================>]   2.28M  2.91MB/s    in 0.8s    

2022-12-06 19:22:57 (2.91 MB/s) - ‘slovnet_ner_news_v1.tar’ saved [2385920/2385920]



In [ ]:
from navec import Navec
from slovnet import NER, Syntax
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import sentenize, tokenize

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)
markup = ner(test_article)
show_markup(markup.text, markup.spans)

На Ставрополье создан Общественный совет при МинэкономразвитияПри 
   LOC────────        ORG───────────────     ORG───────────────── 
министерстве экономического развития Ставропольского края в рамках 
                                     LOC─────────────────          
проекта "открытое правительство" создан Общественный совет, сообщили 
                                        ORG───────────────           
корреспонденту ИА REGNUM в пресс-службе краевого Минэкономразвития 23 
               ORG──────                         ORG──────────────    
июля.Как отметила на организационном заседании совета первый 
заместитель министра Людмила Хохрякова Хохрякова, Общественный совет 
                     PER────────────────────────  ORG─────────────── 
должен будет решать задачи создания общественно-государственного 
партнерства, взаимодействия власти и общества."Нам бы хотелось 
услышать от вас конкретные предложения по улучшению социально-
экономического развития края, по фактическому примене

In [ ]:
pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 74 kB/s 
     |████████████████████████████████| 41 kB 114 kB/s 


In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

doc = Doc(test_article)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)


In [ ]:
doc.sents[1].syntax.print()

                ┌► Его              nmod
              ┌►└─ заместителями    xcomp
┌─────────────└─┌─ стали            
│     ┌─┌─────┌─└► декан            nsubj
│     │ │     │ ┌► экономического   amod
│   ┌─│ │ ┌───└►└─ факультета       nmod
│   │ │ │ │ ┌────► Ставропольского  amod
│   │ │ │ │ │ ┌──► государственного amod
│   │ │ │ │ │ │ ┌► аграрного        amod
│   │ │ │ └►└─└─└─ университета     nmod
│   │ │ └──────►┌─ Ольга            appos
│   │ │         └► Кусакина         flat:name
│   │ │         ┌► и                cc
│ ┌─│ └──────►┌─└─ заместитель      conj
│ │ │         └►┌─ председателя     nmod
│ │ │           └► Совета           nmod
│ │ │           ┌► потребительского amod
│ │ └──────►┌───└─ общества         nmod
│ │         │ ┌──► "                punct
│ │         │ │ ┌► Ставропольские   amod
│ │       ┌─└►└─└─ парки            appos
│ │       │ ┌─└──► культуры         nmod
│ │       │ │   ┌► и                cc
│ │       │ └──►└─ отдыха           conj
│ │       └───